<a href="https://colab.research.google.com/github/Saeif-Saleh/AppPortfolio/blob/master/classificaton_hckr_usrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi
!pip install --upgrade tensorflow


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import scipy.io
import numpy as np

In [0]:
matlab_data=scipy.io.loadmat('/content/drive/My Drive/data_dec_17.mat')

In [0]:
print(matlab_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'c945f', 'c945f_test', 'usrp', 'usrp3_test'])


In [0]:
c945f=matlab_data['c945f']
usrp=matlab_data['usrp']
c945f=c945f[2000000:7000000]
usrp=usrp[2000000:7000000]


In [0]:
c945f=c945f[c945f[:,0]>0]
usrp=usrp[usrp[:,0]>0]

In [0]:
print(c945f)

[[ 0.97690153 -0.0050948 ]
 [ 0.97412139 -0.01266646]
 [ 0.96938461  0.00404707]
 ...
 [ 0.97618783 -0.00727591]
 [ 0.97702587 -0.01606271]
 [ 0.97711927  0.01094663]]


In [0]:
train_x=np.concatenate((c945f,usrp),axis=0)



In [0]:
print(train_x)

[[ 9.76901531e-01 -5.09479642e-03]
 [-9.76597190e-01  3.79039496e-02]
 [ 9.74121392e-01 -1.26664639e-02]
 ...
 [-9.64071393e-01  8.38911533e-03]
 [ 9.65894282e-01  2.12228298e-03]
 [-9.65844631e-01 -9.05364752e-04]]


In [0]:

import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers

In [0]:
train_x=train_x.reshape(-1,2,200)

train_y=np.zeros(len(train_x))
half=int(len(train_y)/2)
train_y[0:half]=0
train_y[half:]=1
train_y=tf.keras.utils.to_categorical(
    train_y,
    num_classes=2,
    dtype='float32'
)

In [0]:
print(train_x.shape)
print(train_y.shape)

(50000, 2, 200)
(50000, 2)


In [0]:
print(train_x[0,:,:])
print(train_y[0,:])

[[ 9.76901531e-01 -5.09479642e-03  9.74121392e-01 -1.26664639e-02
   9.69384611e-01  4.04706597e-03  9.64096487e-01  9.99531150e-03
   9.64494646e-01 -5.88053465e-02  9.70661402e-01  1.10247433e-02
   9.74349022e-01 -2.29752362e-02  9.75798130e-01 -5.25474846e-02
   9.72707808e-01  4.55616415e-02  9.68246341e-01 -1.18595362e-02
   9.64195132e-01  1.09661520e-02  9.65000153e-01  2.30555832e-02
   9.71260607e-01  1.80797875e-02  9.75969017e-01  2.50470638e-03
   9.76839542e-01  2.24419832e-02  9.71983910e-01  2.86539495e-02
   9.68925714e-01 -4.68376577e-02  9.64949727e-01  4.39246297e-02
   9.66918826e-01 -1.72052383e-02  9.69509423e-01  2.91395783e-02
   9.75432873e-01  1.62882507e-02  9.78574395e-01 -1.60538256e-02
   9.73736823e-01  3.02813947e-02  9.68165874e-01 -3.94009203e-02
   9.64747846e-01  1.06303245e-02  9.64436948e-01  5.21925539e-02
   9.68213081e-01 -9.46921110e-03  9.73239124e-01  2.56300569e-02
   9.76918697e-01 -1.40421614e-02  9.73374486e-01  3.36264521e-02
   9.70120

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2, 200)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(2, activation='softmax')
])


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                12832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 130       
Total params: 15,074
Trainable params: 15,074
Non-trainable params: 0
__________________________________________________

In [0]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history=model.fit(train_x, train_y, validation_split=0.2, epochs=20,shuffle=True,batch_size=128)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 2s 41us/sample - loss: 0.6635 - accuracy: 0.6219 - val_loss: 0.9462 - val_accuracy: 0.0000e+00
Epoch 2/20
40000/40000 [==============================] - 1s 28us/sample - loss: 0.6422 - accuracy: 0.6250 - val_loss: 0.9940 - val_accuracy: 0.0000e+00
Epoch 3/20
40000/40000 [==============================] - 1s 28us/sample - loss: 0.5487 - accuracy: 0.6383 - val_loss: 0.6598 - val_accuracy: 0.9998
Epoch 4/20
40000/40000 [==============================] - 1s 27us/sample - loss: 0.4663 - accuracy: 0.7187 - val_loss: 0.5785 - val_accuracy: 0.9699
Epoch 5/20
40000/40000 [==============================] - 1s 28us/sample - loss: 0.4454 - accuracy: 0.7386 - val_loss: 0.5612 - val_accuracy: 0.9340
Epoch 6/20
40000/40000 [==============================] - 1s 29us/sample - loss: 0.4616 - accuracy: 0.7178 - val_loss: 0.7650 - val_accuracy: 0.5744
Epoch 7/20
40000/40000 [========================

In [0]:
c945f_test=matlab_data['c945f_test']
usrp3_test=matlab_data['usrp3_test']
c945f_test=c945f_test[2000000:3000000]
usrp3_test=usrp3_test[2000000:3000000]
# c945f_test=c945f_test[c945f_test[:,0]>0]
# usrp3_test=usrp3_test[usrp3_test[:,0]>0]
test_x=np.concatenate((c945f_test,usrp3_test),axis=0)
test_x=test_x.reshape(-1,2,200)

In [0]:
test_y=np.zeros(len(test_x))
half=int(len(test_y)/2)
test_y[0:half]=0
test_y[half:]=1


In [0]:

test_y=tf.keras.utils.to_categorical(
    test_y,
    num_classes=2,
    dtype='float32')




In [0]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history=model.fit(train_x, train_y, validation_data=(test_x,test_y), epochs=20,shuffle=True,batch_size=128)

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 2s 37us/sample - loss: 0.4573 - accuracy: 0.7747 - val_loss: 4.3072 - val_accuracy: 0.5000
Epoch 2/20
50000/50000 [==============================] - 1s 25us/sample - loss: 0.4537 - accuracy: 0.7751 - val_loss: 3.8245 - val_accuracy: 0.5000
Epoch 3/20
50000/50000 [==============================] - 1s 25us/sample - loss: 0.4480 - accuracy: 0.7797 - val_loss: 3.6734 - val_accuracy: 0.5000
Epoch 4/20
50000/50000 [==============================] - 1s 27us/sample - loss: 0.4367 - accuracy: 0.7893 - val_loss: 4.1769 - val_accuracy: 0.5000
Epoch 5/20
50000/50000 [==============================] - 1s 27us/sample - loss: 0.4338 - accuracy: 0.7917 - val_loss: 3.8371 - val_accuracy: 0.5000
Epoch 6/20
50000/50000 [==============================] - 1s 26us/sample - loss: 0.4409 - accuracy: 0.7852 - val_loss: 3.9265 - val_accuracy: 0.5000
Epoch 7/20
50000/50000 [==============================] 

In [0]:
model.evaluate(test_x,test_y)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[3.980017245991528, 0.5]

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
c945f=matlab_data['c945f']
usrp=matlab_data['usrp']
c945f=c945f[2000000:7000000]
usrp=usrp[2000000:7000000]
train_x=np.concatenate((c945f,usrp),axis=0)

In [0]:
#Reshaping data for the convolutional model
img_rows=200
img_cols=2
train_x = train_x.reshape(-1, img_rows, img_cols, 1)
test_x=test_x.reshape(-1,img_rows,img_cols,1)
input_shape = (img_rows, img_cols, 1)

print('data shape:', train_x.shape) 
train_y=np.zeros(len(train_x))
half=int(len(train_y)/2)
train_y[0:half]=0
train_y[half:]=1
train_y=tf.keras.utils.to_categorical(
    train_y,
    num_classes=2,
    dtype='float32'
)


data shape: (50000, 200, 2, 1)


In [0]:
print(train_x.shape)
print(train_y.shape)

(50000, 200, 2, 1)
(50000, 2)


In [0]:
conv_model = tf.keras.models.Sequential()
conv_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 2,1),data_format='channel_first'))
conv_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
conv_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
conv_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
conv_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
conv_model.add(tf.keras.layers.Flatten())
conv_model.add(tf.keras.layers.Dense(64, activation='relu'))
conv_model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [0]:
conv_model = tf.keras.models.Sequential()
#convolutional layer with rectified linear unit activation
conv_model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',padding="same",
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
conv_model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
conv_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'))
#randomly turn neurons on and off to improve convergence
conv_model.add(tf.keras.layers.Dropout(0.25))
conv_model.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3),
                 activation='relu',padding="same",
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
#randomly turn neurons on and off to improve convergence
conv_model.add(tf.keras.layers.Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
conv_model.add(tf.keras.layers.Flatten())
#fully connected to get all relevant data
conv_model.add(tf.keras.layers.Dense(256, activation='relu'))
#one more dropout for convergence' sake :) 
conv_model.add(tf.keras.layers.Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
conv_model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [0]:
conv_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 2, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 2, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 1, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 1, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 1, 128)       73856     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 1, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)            

In [0]:

conv_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history=conv_model.fit(train_x, train_y, validation_split=0.2, epochs=20,shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 159s 4ms/sample - loss: 0.1954 - accuracy: 0.8880 - val_loss: 1.9313e-07 - val_accuracy: 1.0000
Epoch 2/20
40000/40000 [==============================] - 157s 4ms/sample - loss: 1.0971e-06 - accuracy: 1.0000 - val_loss: 1.1206e-09 - val_accuracy: 1.0000
Epoch 3/20
40000/40000 [==============================] - 151s 4ms/sample - loss: 3.8312e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/20
40000/40000 [==============================] - 151s 4ms/sample - loss: 1.8751e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
40000/40000 [==============================] - 154s 4ms/sample - loss: 3.3935e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
40000/40000 [==============================] - 154s 4ms/sample - loss: 1.6508e-07 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 

KeyboardInterrupt: ignored

In [0]:
#Reshaping data for the convolutional model
img_rows=200
img_cols=2
test_x=test_x.reshape(-1,img_rows,img_cols,1)


print('data shape:', test_x.shape) 
test_y=np.zeros(len(test_x))
half=int(len(test_y)/2)
test_y[0:half]=0
test_y[half:]=1
test_y=tf.keras.utils.to_categorical(
    test_y,
    num_classes=2,
    dtype='float32'
)


data shape: (10000, 200, 2, 1)


In [0]:
conv_model.evaluate(test_x,test_y)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.11106523204110563, 0.9588]

In [0]:
prediction=conv_model.predict_classes(test_x)

In [0]:
for pred,ty in zip(prediction,test_y):
  print(pred,ty)

In [0]:
matlab_data=scipy.io.loadmat('/content/drive/My Drive/23_December_collection.mat')

In [0]:
print(matlab_data.keys())

dict_keys(['__header__', '__version__', '__globals__', 'b585f_test', 'b585f_train', 'c945f_test', 'c945f_train', 'usrp2_test', 'usrp2_train', 'usrp3_test', 'usrp3_train', 'usrp4_test', 'usrp4_train', 'usrp5_test', 'usrp5_train'])


In [0]:
# b585f_train=matlab_data['b585f_train']
# c945f_train=matlab_data['c945f_train']
usrp3_train=matlab_data['usrp3_train']
usrp4_train=matlab_data['usrp4_train']
usrp5_train=matlab_data['usrp5_train']

In [0]:
# b585f_train=b585f_train[2000000:7000000]
# c945f_train=c945f_train[2000000:7000000]
usrp3_train=usrp3_train[2000000:7000000]
usrp4_train=usrp4_train[2000000:7000000]
usrp5_train=usrp5_train[2000000:7000000]

In [0]:
train_x=np.concatenate((usrp3_train,usrp4_train,usrp5_train),axis=0)


In [0]:
train_x=train_x.reshape(-1,2,200)

In [0]:
train_x.shape

(75000, 2, 200)

In [0]:
train_y=np.zeros(len(train_x))
seprator=int(len(train_y)/3)
train_y[0:seprator]=0
train_y[seprator:2*seprator]=1
train_y[2*seprator:3*seprator]=2
# train_y[3*seprator:4*seprator]=3
# train_y[4*seprator:]=4



In [0]:
train_y=tf.keras.utils.to_categorical(
    train_y,
    num_classes=3,
    dtype='float32'
)

In [0]:
img_rows=200
img_cols=2
train_x=train_x.reshape(-1,img_rows,img_cols,1)

In [0]:
conv_model_second = tf.keras.models.Sequential()
#convolutional layer with rectified linear unit activation
conv_model_second.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',padding="same",
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
conv_model_second.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
conv_model_second.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2),padding='same'))
#randomly turn neurons on and off to improve convergence
conv_model_second.add(tf.keras.layers.Dropout(0.25))
conv_model_second.add(tf.keras.layers.Conv2D(128, kernel_size=(3, 3),
                 activation='relu',padding="same",
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
#randomly turn neurons on and off to improve convergence
conv_model_second.add(tf.keras.layers.Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
conv_model_second.add(tf.keras.layers.Flatten())
#fully connected to get all relevant data
conv_model_second.add(tf.keras.layers.Dense(256, activation='relu'))
#one more dropout for convergence' sake :) 
# conv_model_second.add(tf.keras.layers.Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
conv_model_second.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:

conv_model_second.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history=conv_model_second.fit(train_x, train_y, validation_split=0.2, epochs=2,shuffle=True)

Train on 60000 samples, validate on 15000 samples
Epoch 1/2
60000/60000 [==============================] - 228s 4ms/sample - loss: 0.3502 - accuracy: 0.8333 - val_loss: 1.2640 - val_accuracy: 0.0000e+00
Epoch 2/2
60000/60000 [==============================] - 219s 4ms/sample - loss: 0.2131 - accuracy: 0.9013 - val_loss: 0.0106 - val_accuracy: 0.9975


In [0]:
# b585f_test=matlab_data['b585f_test']
# c945f_test=matlab_data['c945f_test']
usrp3_test=matlab_data['usrp3_test']
usrp4_test=matlab_data['usrp4_test']
usrp5_test=matlab_data['usrp5_test']


In [0]:
# c945f_test=c945f_test[2000000:3000000]
# b585f_test=b585f_test[2000000:3000000]
usrp3_test=usrp3_test[2000000:3000000]
usrp4_test=usrp4_test[2000000:3000000]
usrp5_test=usrp5_test[2000000:3000000]

In [0]:
test_x=np.concatenate((usrp3_test,usrp4_test,usrp5_test),axis=0)
img_rows=200
img_cols=2
test_x=test_x.reshape(-1,img_rows,img_cols,1)

In [0]:
test_x.shape

(15000, 200, 2, 1)

In [0]:
test_y=np.zeros(len(test_x))
seprator=int(len(test_y)/3)


In [0]:
test_y[0:seprator]=0
test_y[seprator:2*seprator]=1
test_y[2*seprator:3*seprator]=2
# test_y[3*seprator:4*seprator]=3
# test_y[4*seprator:]=4
test_y=tf.keras.utils.to_categorical(
    test_y,
    num_classes=3,
    dtype='float32'
)

In [0]:
print(test_x.shape)
print(test_y.shape)

(15000, 200, 2, 1)
(15000, 3)


In [0]:
conv_model_second.evaluate(test_x,test_y)

15000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.015188205804799994, 0.99626666]

In [0]:
conv_model_second.evaluate(test_x,test_y)